# Unilever Coding Test

Candidate: Hamidreza Hosseinkhani (`hamidreza@hosseinkhani.me`)

## Damage Calculation
If we name the frontline fighter, `front` and the backup fighter, `back`, we can calculate the total damage with the least arithmetic operations as bellow:

<img width="500" src="https://bayanbox.ir/view/6447323184505277090/20250407-163646.jpg">

In [1]:
def calc_damage(h_front: int, d_front: int,
           h_back: int, d_back: int,
           b: int) -> float:
    return (h_front*(d_front+d_back) + h_back*d_back) / b

## 1. The naive idea: Brute-force

Not practical for large values of N:
- Time Complexity: `O(N²)`
- N in worst case: 500,000
- Operations in worst case: (500,000)² = 250 billion operations
- Took hours or even days on a normal computer

In [2]:
def getMaxDamage(N: int, H: list[int], D: list[int], B: int) -> float:
    max_damage = -1.0
    for front in range(N):
        for back in range(N):
            if back != front:
                back = j
                damage = calc_damage(H[front], D[front], H[back], D[back], B)
                if damage > max_damage:
                    max_damage = damage
    return max_damage

## 2. Prune the search space using heuristics
In many practical scenarios, we can check only a few promising pairs instead of all possible ones. This approach often gets very close to the optimal answer—or even exactly right—but it does not guarantee that the optimal pair will always be found.
- Approximate Alogorithm
- Feasible search space (Only checks up to `K²` pairs instead of `N²`)
- Needs a heuristic function to score promising pairs. A heuristic for choosing top-k frontliners can be `F[i]=H[i]×D[i]` since a high-health fighter with moderate damage can keep the backup fighting longer. For choosing promising backup fighters, we can use he raw `𝐷[𝑖]` score because the backup’s damage is applied for the entire time the front fighter is alive.
- Needs to sort the fighters based on the score that takes `O(Nlog(N))`

In [3]:
def getMaxDamage(N: int, H: list[int], D: list[int], B: int) -> float:

    K = 100 # number of candidates to check

    front_topK_candidates = \
        sorted(range(N), key=lambda i: H[i] * D[i], reverse=True)[:K]
    back_topK_candidates = \
        sorted(range(N), key=lambda i: D[i], reverse=True)[:K]

    max_damage = -1.0
    for front in front_topK_candidates:
        for back in back_topK_candidates:
            if back != front:
                back = j
                damage = calc_damage(H[front], D[front], H[back], D[back], B)
                if damage > max_damage:
                    max_damage = damage
    return max_damage

## 3. Machine Learning
In practical data-driven scenarios where there is enough input/output example, we can train an ML model instead of the heuristic function to model the non-linear interplay between a fighter's health and damage.

## 4. Best Exact Approach: Reformulate the problem as a maximum‐query over a set of linear functions

By transforming the pairing problem into a sequence of linear maximum queries (a well-known technique in computational geometry) and using the `Convex Hull Trick`, we obtain an exact algorithm with a worst-case time complexity of `𝑂(𝑛 log𝑛)`. This is significantly better than brute force and does not rely on any heuristic selection of a candidate set.

- Can be solved using the `convex hull trick`.
- Is provably faster than `𝑂(N²)` brute force in the worst case.


### Find the lines equations

<img width="700" src="https://bayanbox.ir/view/6012393469465137839/20250407-212829.jpg">

Here I use Two-pass `Li Chao Tree` to implement the Convex Hull Trick.

In [4]:
class Line:
    def __init__(self, m, b):
        self.m = m  # slope
        self.b = b  # intercept

    def value(self, x):
        return self.m * x + self.b

In [5]:
class LiChaoTree:
    def __init__(self, lo, hi):
        self.lo = lo
        self.hi = hi
        self.line = None
        self.left = None
        self.right = None

    def _add(self, new, l, r):
        m = (l + r) / 2
        if self.line is None:
            self.line = new
            return

        left_better = new.value(l) > self.line.value(l)
        mid_better = new.value(m) > self.line.value(m)
        right_better = new.value(r) > self.line.value(r)

        if mid_better:
            self.line, new = new, self.line

        if r - l < 1e-6:
            return

        if left_better != mid_better:
            if self.left is None:
                self.left = LiChaoTree(l, m)
            self.left._add(new, l, m)
        elif right_better != mid_better:
            if self.right is None:
                self.right = LiChaoTree(m, r)
            self.right._add(new, m, r)

    def add_line(self, new_line):
        self._add(new_line, self.lo, self.hi)

    def _query(self, x, l, r):
        m = (l + r) / 2
        res_val = self.line.value(x) if self.line else float('-inf')
        if x < m and self.left:
            res_val = max(res_val, self.left._query(x, l, m))
        elif x >= m and self.right:
            res_val = max(res_val, self.right._query(x, m, r))
        return res_val

    def query(self, x):
        return self._query(x, self.lo, self.hi)

In [6]:
def getMaxDamage(N: int, H: list[int], D: list[int], B: int) -> float:
    fighters = [(H[i], D[i]) for i in range(N)]
    min_x = min(H) - 1
    max_x = max(H) + 1

    max_damage = 0.0

    # Process left to right
    tree = LiChaoTree(min_x, max_x)
    for i in range(N):
        h, d = H[i], D[i]
        if i > 0:
            best = tree.query(h)
            total = (h * d + best) / B
            max_damage = max(max_damage, total)
        tree.add_line(Line(d, d * h))

    # Process right to left
    tree = LiChaoTree(min_x, max_x)
    for i in reversed(range(N)):
        h, d = H[i], D[i]
        if i < N - 1:
            best = tree.query(h)
            total = (h * d + best) / B
            max_damage = max(max_damage, total)
        tree.add_line(Line(d, d * h))

    return round(max_damage, 6)

## RUN the code!

In [7]:
if __name__ == '__main__':
    N = int(input())
    H = list(eval(input()))
    D = list(eval(input()))
    B = int(input())

    print(getMaxDamage(N, H, D, B))

4
[1, 1, 2, 3]
[1, 2, 1, 100]
8
62.75
